In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, make_scorer

In [3]:
dataset = pd.read_csv('CKD.csv')

In [4]:
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [5]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)

In [6]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [7]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [9]:
independent= dataset [['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [10]:
dependent=dataset [['classification_yes']]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [12]:
model = SVC(probability=True)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [15]:
from sklearn.model_selection import GridSearchCV
model = SVC(probability=True)

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  
}

scorer = make_scorer(roc_auc_score, needs_proba=True)
grid = GridSearchCV(model, param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 120 candidates, totalling 600 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000], 'degree': [2, 3, 4],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [16]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)   

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}: 0.9924946382275899


In [18]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[51  0]
 [ 1 81]]


In [19]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [20]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0

In [21]:
table=pd.DataFrame.from_dict(re)

In [22]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.278117,0.131248,0.115714,0.042663,0.1,2,scale,linear,"{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'ker...",0.981569,0.962264,1.000000,1.000000,1.000000,0.988767,0.015052,14
1,0.049997,0.006250,0.028127,0.006251,0.1,2,scale,poly,"{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'ker...",0.486532,0.492321,0.477841,0.477841,0.468812,0.480670,0.008084,120
2,0.055027,0.007205,0.034915,0.006436,0.1,2,scale,rbf,"{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'ker...",0.926978,0.981014,0.962573,1.000000,0.981217,0.970356,0.024708,75
3,0.050463,0.007177,0.028126,0.006249,0.1,2,scale,sigmoid,"{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'ker...",0.945100,0.981233,0.962573,0.981217,0.981217,0.970268,0.014509,81
4,0.031251,0.009883,0.028124,0.006252,0.1,2,auto,linear,"{'C': 0.1, 'degree': 2, 'gamma': 'auto', 'kern...",0.981569,0.962264,1.000000,1.000000,1.000000,0.988767,0.015052,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.028126,0.006251,0.025001,0.007656,1000,4,scale,sigmoid,"{'C': 1000, 'degree': 4, 'gamma': 'scale', 'ke...",0.981569,0.943699,0.962573,0.981031,0.981217,0.970018,0.015021,93
116,0.021875,0.007652,0.028127,0.006256,1000,4,auto,linear,"{'C': 1000, 'degree': 4, 'gamma': 'auto', 'ker...",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,57
117,0.122282,0.082422,0.025362,0.007976,1000,4,auto,poly,"{'C': 1000, 'degree': 4, 'gamma': 'auto', 'ker...",0.908877,0.888646,0.851225,0.981217,0.794081,0.884809,0.062045,111
118,0.062472,0.078451,0.022508,0.008481,1000,4,auto,rbf,"{'C': 1000, 'degree': 4, 'gamma': 'auto', 'ker...",0.963284,0.981014,1.000000,0.981031,1.000000,0.985066,0.013807,29


In [23]:
age_input=float(input("Age:"))
bp_input=float(input("BP:"))
al_input=float(input("AL:"))
su_input=float(input("SU:"))
bgr_input=float(input("BGR:"))
bu_input=float(input("BU:"))
sc_input=float(input("SC:"))
sod_input=float(input("SOD:"))
pot_input=float(input("POT:"))
hrmo_input=float(input("HRMO:"))
pcv_input=float(input("PCV:"))
wc_input=float(input("WC:"))
rc_input=float(input("RC:"))
sg_b_input=float(input("Sg B 0 OR 1:"))
sg_c_input=float(input("Sg C 0 OR 1:"))
sg_d_input=float(input("Sg D 0 OR 1:"))
sg_e_input=float(input("Sg E 0 OR 1:"))
rbc_normal_input=float(input("Rbc Normal 0 or 1:"))
pc_normal_input=float(input("Pc Normal 0 or 1:"))
pcc_present_input=float(input("Pcc Present 0 or 1:"))
ba_present_input=float(input("Ba Present 0 or 1:"))
htn_yes_input=float(input("Htn Yes 0 or 1:"))
dm_yes_input=float(input("Dm Yes 0 or 1:"))
cad_yes_input=float(input("Cad Yes 0 or 1:"))
appet_yes_input=float(input("Appet Yes 0 or 1:"))
pe_yes_input=float(input("Pe Yes 0 or 1:"))
ane_yes_input=float(input("Ane Yes 0 or 1:"))

Age: 56
BP: 43
AL: 45
SU: 34
BGR: 22
BU: 56
SC: 44
SOD: 73
POT: 26
HRMO: 34
PCV: 23
WC: 43
RC: 22
Sg B 0 OR 1: 1
Sg C 0 OR 1: 0
Sg D 0 OR 1: 1
Sg E 0 OR 1: 0
Rbc Normal 0 or 1: 1
Pc Normal 0 or 1: 0
Pcc Present 0 or 1: 1
Ba Present 0 or 1: 0
Htn Yes 0 or 1: 1
Dm Yes 0 or 1: 0
Cad Yes 0 or 1: 1
Appet Yes 0 or 1: 0
Pe Yes 0 or 1: 1
Ane Yes 0 or 1: 0


In [24]:
Future_Prediction=grid.predict([[age_input,bp_input,al_input,su_input,bgr_input,bu_input,sc_input,sod_input,pot_input,hrmo_input,pcv_input,wc_input,rc_input,sg_b_input,sg_c_input,sg_d_input,sg_e_input,rbc_normal_input,pc_normal_input,pcc_present_input,ba_present_input,htn_yes_input,dm_yes_input,cad_yes_input,appet_yes_input,pe_yes_input,ane_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [25]:
import pickle
filename="finalized_model_SVC_GridSearchCV.sav"
pickle.dump(grid,open(filename, 'wb'))

In [26]:
preinput=sc.transform([[56,43,45,34,22,56,44,73,26,34,23,43,22,1,0,1,0,1,0,1,0,1,0,1,0,1,0]])

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [27]:
preinput

array([[ 0.24180054, -2.58531895, 35.85900466, 33.93681599, -1.6726475 ,
        -0.0783322 ,  6.87099322, -6.13147338,  9.62750035,  7.96117207,
        -1.96943121, -3.44702237, 21.22464321,  2.02864108, -0.5454915 ,
         2.21610269, -0.13840913,  0.38924947, -1.95866374,  2.91547595,
        -0.23570226,  1.35269628, -0.73926425,  3.41565026, -1.93649167,
         1.91485422, -0.44519456]])

In [28]:
loaded_model=pickle.load(open("finalized_model_SVC_GridSearchCV.sav", 'rb'))
result=loaded_model.predict(preinput)
result                 

array([1])

In [29]:
import numpy as np
def inverse_transform_dummies_with_if(array):
      result = np.array([0])

if np.array_equal(result, np.array([0])):
    print("No")
else:
    print("Yes")

Yes
